This notebook shows how Constant Wavelet Transform can be performed with usage of cmorlet-tensorflow package fork provided by [Fractal Feelings](https://www.kaggle.com/kevinmcisaac) (https://github.com/Kevin-McIsaac/cmorlet-tensorflow/tree/Performance).

Have any questions or suggestions? Please comment below.

**<font color='red'>And if you liked this notebook, please upvote it!</font>**

**Changelog**
* v2 - changed normalization
* v1 - initial version

## Import packages

In [ ]:
!pip install -q git+https://github.com//Kevin-McIsaac/cmorlet-tensorflow@Performance --no-deps

In [ ]:
import numpy as np
import pandas as pd
import os
from glob import glob
import matplotlib.pyplot as plt
from scipy import signal
import tensorflow as tf
from CWT.cwt import ComplexMorletCWT

import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

In [ ]:
num_samples = 4 # first N samples to process
image_size = [256, 256]

## Read training labels

Now we read training labels data, and get npy paths.

In [ ]:
ROOT_DIR = '../input/g2net-gravitational-wave-detection'
df = pd.read_csv(os.path.join(ROOT_DIR, 'training_labels.csv'))
df['path'] = df['id'].apply(lambda x: f'{ROOT_DIR}/train/{x[0]}/{x[1]}/{x[2]}/{x}.npy')

## Demonstrate CWT usage

Let's calculate CWT for 4 first signals with and without usage of a bandpass filter (20-500Hz), and plot results!

In [ ]:
train_files = df['path'].values[:num_samples]
bHP, aHP = signal.butter(8, (20, 500), btype='bandpass', fs=2048)
cwt_transform = ComplexMorletCWT(wavelet_width=8, fs=2048, lower_freq=20, upper_freq=500, n_scales=image_size[0],
                    stride=int(np.ceil(4096/image_size[0])), output='magnitude', data_format='channels_first')
waves = []
cwts = []
waves_f = []
cwts_f = []
for i in range(num_samples):
    wave = np.load(train_files[i])
    wave = wave[0, :]  # for demonstration we will use only signal from one detector
    wave /= tf.math.reduce_max(wave, axis=0, keepdims=True) # normalization
    waves.append(np.squeeze(wave))
    
    # Without a filter
    wave_t = tf.convert_to_tensor(wave[np.newaxis, np.newaxis, :])
    cwt = cwt_transform(wave_t)
    cwts.append(np.squeeze(cwt.numpy()))
    
    # With a filter
    wave_f = wave*signal.tukey(4096, 0.2)
    wave_f = signal.filtfilt(bHP, aHP, wave_f)
    waves_f.append(wave_f)
    wave_t_f = tf.convert_to_tensor(wave_f[np.newaxis, np.newaxis, :])
    cwt_f = cwt_transform(wave_t_f)
    cwts_f.append(np.squeeze(cwt_f.numpy()))

### Without a filter

In [ ]:
fig, axs = plt.subplots(num_samples)
fig.set_figheight(15)
fig.set_figwidth(15)
for i in range(num_samples):
    nid = df['id'][i]
    ntarget = df['target'][i]
    axs[i].title.set_text(f'{nid}.npy, target: {ntarget}')
    axs[i].plot(waves[i])

In [ ]:
fig, axs = plt.subplots(num_samples)
fig.set_figheight(20)
fig.set_figwidth(5)
for i in range(num_samples):
    nid = df['id'][i]
    ntarget = df['target'][i]
    axs[i].title.set_text(f'{nid}.npy, target: {ntarget}')
    axs[i].pcolormesh(cwts[i])

## With a filter

In [ ]:
fig, axs = plt.subplots(num_samples)
fig.set_figheight(15)
fig.set_figwidth(15)
for i in range(num_samples):
    nid = df['id'][i]
    ntarget = df['target'][i]
    axs[i].title.set_text(f'{nid}.npy, target: {ntarget}')
    axs[i].plot(waves_f[i])

In [ ]:
fig, axs = plt.subplots(num_samples)
fig.set_figheight(20)
fig.set_figwidth(5)
for i in range(num_samples):
    nid = df['id'][i]
    ntarget = df['target'][i]
    axs[i].title.set_text(f'{nid}.npy, target: {ntarget}')
    axs[i].pcolormesh(cwts_f[i])

You can use ComplexMorletCWT() as your model block to convert waves to CWT on-the-fly in TensorFlow models.